## AuxTel Image elongation due to azimuth oscillation

Craig Lage 17-Mar-22

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
EfdClient.list_efd_names()

In [ ]:
dir(client)

In [ ]:
await client.get_topics()

In [ ]:
await client.get_fields('lsst.sal.MTMount.logevent_azimuthMotionState')

In [ ]:
# Times to start looking at error codes
start = Time("2023-06-27 12:00:00Z", scale='utc')
end = Time("2023-06-28 12:00:00Z", scale='utc')

In [ ]:
test = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                ['actualPosition'],  start, end)

In [ ]:
len(test)

In [ ]:
# Times to start looking at error codes
start = Time("2023-07-11 12:00:00Z", scale='utc')
end = Time("2023-07-12 12:00:00Z", scale='utc')

In [ ]:
test = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                ['actualPosition'],  start, end)

In [ ]:
len(test)

In [ ]:
test['maxSpeed'].plot()
plt.title('lsst.sal.ESS.airFlow.maxSpeed')
plt.ylabel('m/s')

In [ ]:
# Look before and after the last error time
lastErrorTime = Time(errors.index[3])
before = 300.0
after = 300.0
start = lastErrorTime - TimeDelta(before, format='sec')
end = lastErrorTime + TimeDelta(after, format='sec')
print(start, end)

In [ ]:
status = await client.select_time_series('lsst.sal.ATMCS.logevent_azimuthDrive1Status', \
                                                ['*'],  start, end)

In [ ]:
len(status)

In [ ]:
status

In [ ]:
# Times to start looking at error codes
start = Time("2022-03-17 00:00:00Z", scale='utc')
end = Time("2022-03-17 00:05:00Z", scale='utc')
test = await client.select_packed_time_series('lsst.sal.ATPtg.mountPositions', \
                                                ['*'],  start, end)

In [ ]:
start = Time("2023-06-01T00:00:00Z", scale='utc')
end = Time("2023-06-01T11:00:00Z", scale='utc')
test = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthMotionState', \
                                                ['*'],  start, end)

In [ ]:
len(test)